In [ ]:
# seam line group 이름 생성
stitch_dict.keys()
seam_line_groups = [f"SeamLinePairGroup_{idx}" for idx, _ in enumerate(stitch_dict.keys())]
print(seam_line_groups)

In [ ]:
# Seam Line Group 이름, pair 정보
SeamLinePairGroup = []
for idx, stitch in enumerate(stitch_dict.values()):
    SeamLinePairGroup.append(f"SeamLinePairGroup_{idx}")
print(SeamLinePairGroup)

['SeamLinePairGroup_0', 'SeamLinePairGroup_1', 'SeamLinePairGroup_2', 'SeamLinePairGroup_3', 'SeamLinePairGroup_4', 'SeamLinePairGroup_5', 'SeamLinePairGroup_6', 'SeamLinePairGroup_7', 'SeamLinePairGroup_8', 'SeamLinePairGroup_9', 'SeamLinePairGroup_10', 'SeamLinePairGroup_11', 'SeamLinePairGroup_12', 'SeamLinePairGroup_13', 'SeamLinePairGroup_14', 'SeamLinePairGroup_15', 'SeamLinePairGroup_16', 'SeamLinePairGroup_17', 'SeamLinePairGroup_18', 'SeamLinePairGroup_19', 'SeamLinePairGroup_20', 'SeamLinePairGroup_21', 'SeamLinePairGroup_22', 'SeamLinePairGroup_23']


In [ ]:
#TODO: 각 패널 엣지에 x, y 에 대한 좌표 정보를 얻기 위한 함수
import pandas as pd

def process_path_data(data):
    """
    주어진 Path 데이터를 DataFrame으로 처리.
    - 요소 타입 변환 (QuadraticBezier -> Bezier Curve, Line -> Straight)
    - start, control, control1, control2, end, radius의 실수부/허수부를 x, y로 분리
    - 복소수 열 제거
    
    Args:
        data (tuple): Path 객체와 속성 정보가 포함된 데이터
    
    Returns:
        pd.DataFrame: 처리된 데이터프레임
    """
    rows = []
    # Path 내부 요소 순회
    for element in data[0]:
        row = {
            "type": type(element).__name__,  # 요소 타입
            "start": getattr(element, "start", None),
            "controla": getattr(element, "control", None),
            "controlb": getattr(element, "control", None),
            "control1": getattr(element, "control1", None),
            "control2": getattr(element, "control2", None),
            "end": getattr(element, "end", None),
            "radius": getattr(element, "radius", None),
        }
        rows.append(row)
    
    # DataFrame 생성
    df = pd.DataFrame(rows)
    
    # type 열의 값을 변경하기 위한 매핑 사전
    type_mapping = {
        "QuadraticBezier": "Bezier Curve",
        "CubicBezier": "Bezier Curve",
        "Line": "Straight",
        "Arc": "Straight", #임의로 바꿈    
    }
    
    # type 열 값 변경
    df["type"] = df["type"].map(type_mapping).fillna(df["type"])
    
    # 대상 열 목록
    coordinate_columns = ["start",  "controla", "controlb","control1", "control2", "radius", "end"] 
    
    # 실수부와 허수부를 분리하여 새로운 열로 추가하는 함수
    def extract_coordinates(df, columns):
        for col in columns:
            df[f"{col}_x"] = df[col].apply(lambda z: z.real if isinstance(z, complex) else None)
            df[f"{col}_y"] = df[col].apply(lambda z: z.imag if isinstance(z, complex) else None)
        # TODO: y 좌표를 상하 반전
        max_y = df[['start_y', 'controla_y', 'controlb_y', 'control1_y', 'control2_y', 'end_y']].max().max()
        min_y = df[['start_y', 'controla_y', 'controlb_y', 'control1_y', 'control2_y', 'end_y']].min().min()
        stand_y = (max_y + min_y)
        for col in columns: 
            df[f"{col}_y"] = stand_y - df[f"{col}_y"]
        df['controlb_x'] = df['controla_x'].round(4) # TODO: ontrolb_x 에 round
        # TODO: 여기까지
        return df
    
    # 실수부와 허수부를 분리하여 데이터프레임에 추가
    df = extract_coordinates(df, coordinate_columns)
    
    # 기존 열 중 복소수 값을 제거
    df = df.drop(columns=coordinate_columns)
    
    return df

In [ ]:
# panel 이름 , edge 별 type, x, y 좌표, 
panel_names = list(pattern.panel_order())
for idx, panel_name in enumerate(panel_names):
    print(process_path_data(drawn_pattern_list[idx]))
    print(panel_name)

            type    start_x    start_y  control_x  control_y  control1_x  \
0   Bezier Curve  23.838050  37.467706  20.415804  37.467706         NaN   
1       Straight  17.003877  37.660943        NaN   0.000000         NaN   
2       Straight  15.351117  25.771222        NaN   0.000000         NaN   
3   Bezier Curve  15.228284  37.774638  12.956118  37.936948         NaN   
4       Straight  10.688537  38.185092        NaN   0.000000         NaN   
5       Straight   8.555268  27.594162        NaN   0.000000         NaN   
6   Bezier Curve   8.921332  38.391598   4.806185  38.960192         NaN   
7       Straight   0.711692  39.836598        NaN   0.000000         NaN   
8       Straight   0.000000  25.602731        NaN   0.000000         NaN   
9    CubicBezier   0.000000  20.989281        NaN   0.000000    7.776448   
10      Straight   6.609650   2.547801        NaN   0.000000         NaN   
11           Arc  13.019250   0.000000        NaN   0.000000         NaN   
12      Stra

In [ ]:
### 원래꺼 
# 이게 2024.01.14 12:58 + (15:55 기준 최신본.
# TODO: Pattern List + (InstanceDataList)에 대한 정보를 json으로 넣는 코드.
# ++ Bezier Curve, Straight가 아닌
# Arc , Qudratic Curve, Cubic Curve에 대해서는 어떻게 다룰지. 
# InstanceDataList 생성

# InstanceDataList 추가 루프

import pandas as pd
import json
import math

# 패널 데이터프레임 생성
panel_dataframes = [process_path_data(drawn_pattern) for drawn_pattern in drawn_pattern_list]
panel_names = list(pattern.panel_order())

# Initialize the top-level JSON structure
garment_data = {
    "FabricList": [],
    "GradingRuleTableList": [],
    "PatternList": [],
    "SymmetricDataList": [],
    "InstanceDataList": [],
    "SeamLinePairGroupList": [],
}

# Process each panel
for panel_name, df in zip(panel_names, panel_dataframes):
    # Initialize JSON structure for the panel
    panel_data = {
        "Name": panel_name,
        "fGrainlineAngle": 0.0,
        "ID": panel_name,
        "strSuperImposeSide": "None",
        "CurrentFabricUUID": "DWX6ey",
        "IsClosed": False,
        "InternalLineList": [],
        "ButtonHeadList": [],
        "ButtonHoleList": [],
        "AnnotationList": [],
        "NotchList": [],
        "IsHalfSymmetric": False,
        "ShapeInfo": {
            "IsSlashed": False,
            "LineList": [],
            "HalfSymmetryPointIDMap": {},
            "HalfSymmetryLineIDMap": {}
        },
        "ArrangementPointDataMap": {
                "PointName": "Body_Front_Center_3",
                "fOffSetX": 0.0,
                "fOffSetY": 0.05,
                "fAngle": 180.0
        }
    }
    # InstanceDataList에 json에 추가
    instance_data = {
        "OriginPatternID": panel_name,  # 패널 이름을 OriginPatternID로 사용
        "InstancePatternIDArray": []   # 빈 리스트로 추가
    }
    garment_data["InstanceDataList"].append(instance_data)
    # 패널별 좌표 ID 초기화
    coordinate_id_map = {}
    point_counter = 0  # 패널 내 좌표 카운터

    # Generate LineList and PointList for the panel
    for edge_idx, row in df.iterrows():
        line_id = f"{panel_name}_edge_{edge_idx}"
        point_list = []
        if row["type"] == "Bezier Curve":
            row_type = ["Straight", "Bezier Curve", "Bezier Curve", "Straight"]
            idx = 0
            for col_x, col_y in zip(row.index[1::2], row.index[2::2]):  # Skip 'type' and go by pairs
                x, y = row[col_x], row[col_y]
                if not (pd.isna(x) or pd.isna(y)):#TODO: or math.isclose(x, 0.0) and math.isclose(y, 0.0) ):
                    # 좌표에 대해 새로운 ID 생성 (패널 내에서만)
                    key = (x, y)
                    if key not in coordinate_id_map:
                        coordinate_id_map[key] = f"{panel_name}_point_{point_counter}"
                        point_counter += 1
                    point_id = coordinate_id_map[key]
                    
                    point_list.append({
                        "ID": point_id, 
                        "PointType": row_type[idx],
                        "Position": {"x": x, "y": y},
                        "GradingRuleID": -1
                    })
                    idx +=1        
        if row["type"] == "Straight":
            # Iterate through columns 2 by 2 for x and y
            for col_x, col_y in zip(row.index[1::2], row.index[2::2]):  # Skip 'type' and go by pairs
                x, y = row[col_x], row[col_y]
                if not (pd.isna(x) or pd.isna(y)):#TODO: or math.isclose(x, 0.0) and math.isclose(y, 0.0) ):
                    # 좌표에 대해 새로운 ID 생성 (패널 내에서만)
                    key = (x, y)
                    if key not in coordinate_id_map:
                        coordinate_id_map[key] = f"{panel_name}_point_{point_counter}"
                        point_counter += 1
                    point_id = coordinate_id_map[key]
                    
                    point_list.append({
                        "ID": point_id,
                        "PointType": row["type"],
                        "Position": {"x": x, "y": y},
                        "GradingRuleID": -1
                    })

        # Add Line to LineList
        line = {"ID": line_id, "PointList": point_list}
        panel_data["ShapeInfo"]["LineList"].append(line)

    # Add the panel data to the PatternList
    garment_data["PatternList"].append(panel_data)

# Convert to JSON string with indentation
json_output = json.dumps(garment_data, indent=4)

# Save to a file (optional)
output_file = "multiple_panels_data3.json"
with open(output_file, "w") as file:
    file.write(json_output)

# 결과 출력
print(json_output)

In [ ]:
# 각 panel에 대한 start, end 좌표를 저장해 놓는 함수
import math
from svgpathtools import Path, Line, QuadraticBezier, CubicBezier, Arc

def calculate_edge_length(edge):
    """
    각 엣지의 길이를 계산.
    """
    if isinstance(edge, Line):
        return abs(edge.end - edge.start)
    elif isinstance(edge, QuadraticBezier):
        return edge.length()
    elif isinstance(edge, CubicBezier):
        return edge.length()
    elif isinstance(edge, Arc):
        return edge.length()
    else:
        raise ValueError(f"Unsupported edge type: {type(edge)}")

def calculate_fstart_fend(path):
    """
    전체 엣지에 대해 fstart와 fend를 계산.
    """
    total_length = sum(calculate_edge_length(edge) for edge in path)
    fstart = 0.0
    results = []

    for idx, edge in enumerate(path):  # enumerate를 사용하여 번호 부여
        edge_length = calculate_edge_length(edge)
        fend = fstart + (edge_length / total_length)
        results.append({
            "edge_id": idx,  # 엣지 번호
            "edge": str(edge),  # Edge를 문자열로 변환하여 저장
            "fstart": round(fstart, 6),
            "fend": round(fend, 6)
        })
        fstart = fend

    return results

# 여러 패널의 정보를 저장할 딕셔너리
panel_fstart_fend_info = {}

# 패널 리스트 (예: pattern.panel_order())
panel_list = pattern.panel_order()

# 각 패널에 대해 fstart, fend 계산
for panel_name in panel_list:
    path = panel_svg_path_dict[panel_name][0]  # 패널에 해당하는 경로 정보 가져오기
    fstart_fend_results = calculate_fstart_fend(path)
    panel_fstart_fend_info[panel_name] = fstart_fend_results  # 결과를 딕셔너리에 저장

# panel_fstart_fend_info에 패널 정보, edge, start, end, edge_id 까지 저장 되어있다.
# 결과 출력 (딕셔너리 형태)
import pprint
pprint.pprint(panel_fstart_fend_info)

{'left_btorso': [{'edge': 'Line(start=37.46770632525742j, '
                          'end=10.034413661707j)',
                  'edge_id': 0,
                  'fend': 0.175079,
                  'fstart': 0.0},
                 {'edge': 'Arc(start=10.034413661707j, '
                          'radius=(10.849457592383715+10.849457592383715j), '
                          'rotation=0, large_arc=False, sweep=False, '
                          'end=(10.818800000000003+0j))',
                  'edge_id': 1,
                  'fend': 0.278636,
                  'fstart': 0.175079},
                 {'edge': 'Line(start=(10.818800000000003+0j), '
                          'end=(17.2284+2.5478006069423884j))',
                  'edge_id': 2,
                  'fend': 0.322656,
                  'fstart': 0.278636},
                 {'edge': 'CubicBezier(start=(17.2284+2.5478006069423884j), '
                          'control1=(16.844929+13.09047060694239j), '
                          'contr

In [ ]:
# 2024.01.14 14:33 기준 2:51 기준
# TODO: Seam Line Pair Group json으로 넣는 코드
# SeamLinePairGroupList 생성
garment_data["SeamLinePairGroupList"] = []
fabric_group = {
    "FabricName" : "Default Fabric",
    "FabricType" : "None",
    "FabricContent" : "None",
    "strBaseColorHexCode" : "#FFFFFF",
    "FabricUUID" : "DWX6ey"
    }
# SeamLinePairGroupList 생성 루프
for idx, value in enumerate(stitch_dict.values()):
    # "Name" 생성
    group_name = f"SeamLineGroup_{idx}"

    # value에서 첫 번째와 두 번째 ShapeID 정보 추출
    first_info = value[0]
    second_info = value[1]

    # 첫 번째 정보
    first_panel = first_info["panel"]
    first_edge = first_info["edge"]
    first_shape_id = f"{first_panel}_edge_{first_edge}"

    # 첫 번째 fStart와 fEnd 추출
    first_fstart_fend = next(
        (item for item in panel_fstart_fend_info[first_panel] if item["edge_id"] == first_edge),
        None
    )
    first_fstart = first_fstart_fend["fstart"]
    first_fend = first_fstart_fend["fend"]

    # 두 번째 정보
    second_panel = second_info["panel"]
    second_edge = second_info["edge"]
    second_shape_id = f"{second_panel}_edge_{second_edge}"

    # 두 번째 fStart와 fEnd 추출
    second_fstart_fend = next(
        (item for item in panel_fstart_fend_info[second_panel] if item["edge_id"] == second_edge),
        None
    )
    second_fstart = second_fstart_fend["fstart"]
    second_fend = second_fstart_fend["fend"]
    # Fabric default 구조 생성

    # JSON 구조 생성
    seam_line_pair_group = {
        "Name": group_name,
        "bIsTurned": False,
        "PairList": [
            {
                "First": {
                    "ShapeID": first_shape_id,
                    "LengthParam": {
                        "fStart": first_fstart,
                        "fEnd": first_fend
                    },
                    "Direction": True
                },
                "Second": {
                    "ShapeID": second_shape_id,
                    "LengthParam": {
                        "fStart": second_fstart,
                        "fEnd": second_fend
                    },
                    "Direction": True # TODO True로 일단 default
                }
            }
        ],
        "FoldData": {
            "iAngle": 180,
            "iStrength": 5
        }
    }

    # SeamLinePairGroupList에 추가
    garment_data["SeamLinePairGroupList"].append(seam_line_pair_group)

garment_data["FabricList"].append(fabric_group)    

# JSON 출력
json_output = json.dumps(garment_data, indent=4)

# Save to file (optional)
output_file = "garment_with_seam_lines.json"
with open(output_file, "w") as file:
    file.write(json_output)

# Print the result
print(json_output)

In [ ]:
# InstanceDataList json 넣는 코드

In [ ]:
# 젤 마지막에 진행 해야 할 부분.
# Initialize the top-level JSON structure
garment_data = {
    "FabricName": [],
    "GradingRuleTableList": [],
    "PatternList": [],
    "SymmetricDataList": [],
    "InstanceDataList": [],
    "SeamLinePairGroupList": [],
}

# 1. FabricName 정보 추가
garment_data["FabricName"] = ["DefaultFabric"]  # 여러 개의 패브릭 이름을 넣을 수 있습니다.

# 2. GradingRuleTableList 정보 추가
garment_data["GradingRuleTableList"] = [
    {
        "ID": "Rule1",
        "Description": "Default Grading Rule",
        "Details": []
    }
]

# 3. PatternList 정보 추가 (기존 코드 사용)
for panel_name, df in zip(panel_names, panel_dataframes):
    panel_data = {
        "Name": panel_name,
        "fGrainlineAngle": 0.0,
        "ID": panel_name,
        "strSuperImposeSide": "None",
        "CurrentFabricUUID": "",
        "IsClosed": False,
        "InternalLineList": [],
        "ButtonHeadList": [],
        "ButtonHoleList": [],
        "AnnotationList": [],
        "NotchList": [],
        "IsHalfSymmetric": False,
        "ShapeInfo": {
            "IsSlashed": False,
            "LineList": [],
            "HalfSymmetryPointIDMap": {},
            "HalfSymmetryLineIDMap": {}
        },
        "ArrangementPointDataMap": {
                "PointName": "Body_Front_Center_3",
                "fOffSetX": 0.0,
                "fOffSetY": 0.05,
                "fAngle": 0.0
        }
    }

    coordinate_id_map = {}
    point_counter = 0
    for edge_idx, row in df.iterrows():
        line_id = f"{panel_name}_edge_{edge_idx}"
        point_list = []
        for col_x, col_y in zip(row.index[1::2], row.index[2::2]):
            x, y = row[col_x], row[col_y]
            if not (pd.isna(x) or pd.isna(y) or math.isclose(x, 0.0) and math.isclose(y, 0.0)):
                key = (x, y)
                if key not in coordinate_id_map:
                    coordinate_id_map[key] = f"{panel_name}_point_{point_counter}"
                    point_counter += 1
                point_id = coordinate_id_map[key]
                point_list.append({
                    "ID": point_id,
                    "PointType": row["type"],
                    "Position": {"x": x, "y": y},
                    "GradingRuleID": -1
                })
        line = {"ID": line_id, "PointList": point_list}
        panel_data["ShapeInfo"]["LineList"].append(line)
    garment_data["PatternList"].append(panel_data)

# 4. SymmetricDataList 정보 추가
garment_data["SymmetricDataList"] = [
    {
        "ID": "Symmetry1",
        "Type": "Horizontal",
        "ReferencePanel": panel_names[0] if panel_names else None
    }
]

# 5. InstanceDataList 정보 추가
garment_data["InstanceDataList"] = [
    {
        "InstanceID": "Instance1",
        "PanelReference": panel_names[0] if panel_names else None,
        "Position": {"x": 0.0, "y": 0.0, "z": 0.0},
        "Rotation": {"x": 0.0, "y": 0.0, "z": 0.0}
    }
]

# 6. SeamLinePairGroupList 정보 추가
garment_data["SeamLinePairGroupList"] = [
    {
        "GroupID": "Seam1",
        "SeamLines": [
            {"StartPointID": "point_1", "EndPointID": "point_2"},
            {"StartPointID": "point_3", "EndPointID": "point_4"}
        ]
    }
]

# Convert to JSON string with indentation
json_output = json.dumps(garment_data, indent=4)

# Save to a file (optional)
output_file = "complete_garment_data.json"
with open(output_file, "w") as file:
    file.write(json_output)

# 결과 출력
print(json_output)